In [11]:
from datetime import datetime, timedelta
from pathlib import Path
import os
import sys
import numpy as np
import pandas as pd
import re
import AMBRA_Backups
from bs4 import BeautifulSoup
import AMBRA_Utils
from redcap_funcs import comp_schema_cap_db, get_project_schema

db_name = 'CAPTIVA'
db = AMBRA_Backups.database.Database(db_name)
project_name = 'CAPTIVA DC'
project = AMBRA_Backups.redcap_funcs.get_redcap_project(project_name) 

def q(query, record=None):
    if any(substring in query for substring in ['UPDATE', 'INSERT', 'DELETE']):
        return db.run_insert_query(query, record)
    elif 'SELECT' in query:
        return pd.DataFrame(db.run_select_query(query, record, column_names=True))
    else:
        print('You dont have UPDATE, INSERT, DELETE or SELECT in your query my guy')

def display_md(df):
    return print(df.to_markdown())

In [14]:
df = pd.DataFrame({
    'a': ['1', '1', '1']
})
df['a']

0    1
1    1
2    1
Name: a, dtype: object

In [13]:
AMBRA_Backups.redcap_funcs.grab_logs(db, project, True, start_date=datetime(2000, 1,1))

[{'timestamp': '2024-02-05 12:41',
  'username': 'znaddb0p',
  'action': 'Create record 9999',
  'details': "q1006(1) = checked, q1006(2) = checked, baseline_brain_crf_complete = '0', record_id = '9999'",
  'record': '9999'},
 {'timestamp': '2024-02-05 16:40',
  'username': 'znaddb0p',
  'action': 'Update record 9999',
  'details': "baseline_brain_v2_crf_complete = '0'",
  'record': '9999'},
 {'timestamp': '2024-02-06 11:39',
  'username': 'znaddb0p',
  'action': 'Update record 9999',
  'details': 'q1008_r_car_it(1) = checked, q1008_l_car_it(1) = checked',
  'record': '9999'},
 {'timestamp': '2024-02-06 11:57',
  'username': 'znaddb0p',
  'action': 'Update record 9999',
  'details': '',
  'record': '9999'},
 {'timestamp': '2024-02-06 16:02',
  'username': 'zmalth9v',
  'action': 'Create record 9998',
  'details': "form_1_complete = '0', record_id = '9998'",
  'record': '9998'},
 {'timestamp': '2024-02-07 09:13',
  'username': 'zmalth9v',
  'action': 'Update record 9999',
  'details': "

In [9]:
## schema comp sanity check

db = AMBRA_Backups.database.Database(db_name)
project = AMBRA_Backups.redcap_funcs.get_redcap_project(project_name)

forms = [f['instrument_name'] for f in project.export_instruments()]


master_discreps = ''

for crf_name in forms:

    # redcap_variable discrepancies
    unique_data_vars = pd.DataFrame(db.run_select_query("""SELECT DISTINCT(redcap_variable) 
        FROM CRF_RedCap
        JOIN CRF_Data_RedCap
            ON CRF_RedCap.id = CRF_Data_RedCap.id_crf
        WHERE crf_name = %s""", [crf_name], column_names=True))
    var_discrep_string = ''
    if not unique_data_vars.empty:
        unique_data_vars = unique_data_vars['redcap_variable']
        schema_vars = pd.DataFrame(db.run_select_query("""SELECT redcap_variable FROM CRF_Schema_RedCap
            WHERE crf_name = %s""", [crf_name], column_names=True))['redcap_variable']

        var_discreps = unique_data_vars[~unique_data_vars.isin(schema_vars)].to_list()
        if var_discreps:
            # redcap_variables inside the data table might not have a schema variable to coorispond to, but might have an active crf_id
            # So the non-included redcap_variable will be attached to a csv report if not taken out of the data table, or have the schema corrected. Case by case 
            var_discrep_string = f"\nThe following CRF_Data_RedCap.redcap_variable's are not in CRF_schema_RedCap.redcap_variable's:\n{var_discreps}\n\n"

    print('redcap_variables')
    print('CRF_Data_RedCap')
    display(unique_data_vars.to_frame())
    print('CRF_Schema_RedCap')
    display(schema_vars.to_frame())



    # question text discrepancies
    schema_questions = pd.DataFrame(db.run_select_query("""SELECT question_text, redcap_variable FROM CRF_Schema_RedCap
                                    WHERE crf_name = %s AND question_text IS NOT NULL""", [crf_name], column_names=True))
    schema_questions['variable-value'] = schema_questions['redcap_variable']+schema_questions['question_text']

    api_questions = pd.DataFrame(project.export_metadata())
    api_questions = api_questions[api_questions['form_name'] == crf_name]
    field_names = pd.DataFrame(project.export_field_names())
    field_names.rename(columns={'original_field_name': 'field_name'}, inplace=True)
    # api_questions = pd.merge(field_names, api_questions, on='field_name')
    api_questions = pd.merge(api_questions, field_names, on='field_name', how='left')

    # replacing
    def val_to_text(row):
        if row['field_type'] == 'checkbox':
            dic = {op.split(',')[0].strip() : op.split(',')[1].strip() for op in row['select_choices_or_calculations'].split('|')}
            return dic[row['choice_value']]
        else:
            return row['select_choices_or_calculations']
    api_questions['select_choices_or_calculations'] = api_questions.apply(val_to_text, axis=1)

    def replace_seperators(row):
        if row['field_type'] == 'radio':
            return row['select_choices_or_calculations'].replace(',', '=')
        else:
            return row['select_choices_or_calculations']
    api_questions['select_choices_or_calculations'] = api_questions.apply(replace_seperators, axis=1)

    api_questions.loc[(api_questions['field_type'] == 'checkbox') | 
                    (api_questions['field_type'] == 'radio') | 
                    (api_questions['field_type'] == 'yesno'), 'data_type'] = 'int'
    api_questions.loc[api_questions['field_type'] == 'text', 'data_type'] = 'string'

    api_questions.loc[api_questions['export_field_name'].str.contains('___', na=False), 'export_field_name'] = api_questions['export_field_name'].apply(lambda x: 
                                                                                                        x.split('___')[0] if isinstance(x, str) else x)+'('+api_questions['choice_value']+')'
    api_questions['redcap_variable'] = api_questions['export_field_name']
    api_questions

    def only_html(row):
        soup = BeautifulSoup(row['field_label'], 'html.parser')
        if bool(soup.find()):
            return row['field_label']
    master_html = ''.join(api_questions.apply(only_html, axis=1).dropna().values.tolist())
    api_questions = api_questions[(api_questions['field_type'] != 'descriptive') &
                                    ~(api_questions['field_label'].str.contains('record', case=False)) &
                                (~api_questions['field_name'].apply(lambda x: x in master_html))][['redcap_variable','field_label']]
    api_questions['variable-value'] = api_questions['redcap_variable']+api_questions['field_label']

    question_discreps = api_questions[~api_questions['variable-value'].isin(schema_questions['variable-value'])]
    ques_discrep_string = ''
    if not question_discreps.empty:
        discrep_dict = {v[0]:v[1] for v in question_discreps.values}
        ques_discrep_string = f"\nThe following api-metadata question_text's are not in CRF_Schema_RedCap.question_text's:\n{{redcap_variable : question_text}}\n\n{discrep_dict}\n\n"  

    print('question_text')
    print('schema_questions')
    display(schema_questions)
    print('api_questions')
    display(api_questions.reset_index())


    # radio button option discrepancies
    schema_radio_options = pd.DataFrame(db.run_select_query("""SELECT * FROM CRF_Schema_RedCap
                                WHERE crf_name = %s AND question_type = 'radio'""", [crf_name], column_names=True))
    radio_discrep_string = ''
    if not schema_radio_options.empty:
        schema_radio_options = schema_radio_options['data_labels']
        def schema_rep_seps(string_ops):
            return '|'.join([ss.split('=')[0].strip()+'='+'='.join(ss.split('=')[1:]).strip() for ss in string_ops.split('|')])
        schema_radio_options = schema_radio_options.apply(schema_rep_seps)

        api_radio_options = pd.DataFrame(project.metadata)
        api_radio_options = api_radio_options[(api_radio_options['form_name'] == crf_name) & 
                                            (api_radio_options['field_type'] == 'radio')][['field_name', 'select_choices_or_calculations']]

        def api_rep_seps(string_ops):
            return '|'.join([ss.split(',')[0].strip()+'='+','.join(ss.split(',')[1:]).strip() for ss in string_ops.split('|')])
        api_radio_options['select_choices_or_calculations'] = api_radio_options['select_choices_or_calculations'].apply(api_rep_seps)

        radio_discreps = api_radio_options[~api_radio_options['select_choices_or_calculations'].isin(schema_radio_options)]
        if not radio_discreps.empty:
            discrep_dict = {v[0]:v[1] for v in radio_discreps.values}
            radio_discrep_string = f"The following api-metadata radio button options's are not in CRF_Schema_RedCap.data_labels's(radio button options):\n{{redcap_variable : select_choices_or_calculations}}\n\n{discrep_dict}\n"

    print('radio button options')
    print('schema_radio_options')
    display(schema_radio_options.to_frame())
    print('api_radio_options')
    display(api_radio_options.reset_index())

    form_discrepancies = var_discrep_string + ques_discrep_string + radio_discrep_string
    if form_discrepancies:
        master_discreps += f'\n{crf_name:-^{40}}\n{form_discrepancies}'


if master_discreps:
    print('====================================================================')
    print('====================================================================')
    print(master_discreps)
    print('====================================================================')
    print('====================================================================')
    raise Exception('Please handle the above discrepancies')

redcap_variables
CRF_Data_RedCap


,redcap_variable
0,base_brain_status
1,base_vasc_status
2,form_1_complete


CRF_Schema_RedCap


,redcap_variable
0,base_brain_status
1,base_vasc_status
2,form_1_complete
3,record_id


question_text
schema_questions


,question_text,redcap_variable,variable-value
0,Baseline Brain Status,base_brain_status,base_brain_statusBaseline Brain Status
1,Baseline Vasculature Status,base_vasc_status,base_vasc_statusBaseline Vasculature Status


api_questions


,index,redcap_variable,field_label,variable-value
0,1,base_brain_status,Baseline Brain Status,base_brain_statusBaseline Brain Status
1,2,base_vasc_status,Baseline Vasculature Status,base_vasc_statusBaseline Vasculature Status


radio button options
schema_radio_options


AttributeError: 'DataFrame' object has no attribute 'to_frame'

In [45]:
crf_name = 'mr'

schema_questions = pd.DataFrame(db.run_select_query("""SELECT question_text, redcap_variable FROM CRF_Schema_RedCap
                                WHERE crf_name = %s AND question_text IS NOT NULL""", [crf_name], column_names=True))
schema_questions['variable-value'] = schema_questions['redcap_variable']+schema_questions['question_text']

api_questions = pd.DataFrame(project.export_metadata())
api_questions = api_questions[api_questions['form_name'] == crf_name]
field_names = pd.DataFrame(project.export_field_names())
field_names.rename(columns={'original_field_name': 'field_name'}, inplace=True)
# api_questions = pd.merge(field_names, api_questions, on='field_name')
api_questions = pd.merge(api_questions, field_names, on='field_name', how='left')
api_questions

,field_name,form_name,section_header,field_type,field_label,select_choices_or_calculations,field_note,text_validation_type_or_show_slider_number,text_validation_min,text_validation_max,identifier,branching_logic,required_field,custom_alignment,question_number,matrix_group_name,matrix_ranking,field_annotation,choice_value,export_field_name
0,q2001,mr,,radio,Time point,"1, Baseline SOC | 2, Follow-Up | 3, Unscheduled",,,,,,,,RH,,,,"@DEFAULT=""1""",,q2001
1,q2002,mr,,checkbox,Scan Type,"1, MRI | 2, MRA | 3, MRP | 4, Others",,,,,,,,RH,,,,,1,q2002___1
2,q2002,mr,,checkbox,Scan Type,"1, MRI | 2, MRA | 3, MRP | 4, Others",,,,,,,,RH,,,,,2,q2002___2
3,q2002,mr,,checkbox,Scan Type,"1, MRI | 2, MRA | 3, MRP | 4, Others",,,,,,,,RH,,,,,3,q2002___3
4,q2002,mr,,checkbox,Scan Type,"1, MRI | 2, MRA | 3, MRP | 4, Others",,,,,,,,RH,,,,,4,q2002___4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,q2129,mr,,text,Signature of Central Reader,,,,,,,,,RH,,,,,,q2129
367,q2131,mr,,notes,Reviewer's general comments,,,,,,,,,RH,,,,,,q2131
368,mr_status,mr,"<div class=""rich-text-field-label""><p><strong>...",radio,CRF status,"1, Unread | 2, Query generated | 3, Read - Unv...",,,,,,,,,,,,@IF([user-role-label] = 'Admin' or user-role-l...,,mr_status
369,mri_comments,mr,,notes,CRP comments,,,,,,,,,,,,,@IF([user-role-label] = 'Admin' or user-role-l...,,mri_comments


In [63]:

patient_name = '11003'
patient_id = '6037'
instance = 1
verified = ''

form_df = AMBRA_Backups.redcap_funcs.get_form_df(project, patient_name, crf_name, instance)
# if form_df.empty: # if empty, means there is no live data for this patient and a deleted log should appear later
#     continue 
# crf_id = db.run_insert_query(f"""INSERT INTO CRF_RedCap (id_patient, crf_name, instance, verified, deleted) VALUES 
#                                 (%s, %s, {'NULL' if instance is None else instance}, %s, %s)""", [patient_id, crf_name, verified, deleted])

irr_cols = 3 if form_df.columns[1] == 'redcap_repeat_instrument' else 1 # number of irrelevant fields ie. record_id, redcap_repeat_instrument, redcap_repeat_instance
form_df = form_df[form_df.columns[irr_cols:]]
form_df = form_df.melt(var_name='redcap_variable')
form_df.loc[form_df['redcap_variable'].str.contains('___'), 'redcap_variable'] = form_df['redcap_variable']+')'
form_df.loc[form_df['redcap_variable'].str.contains('___'), 'redcap_variable'] = form_df['redcap_variable'].str.replace('___', '(')

def under_to_neg(row):
    redcap_variable = row['redcap_variable']
    if '(' in redcap_variable and ')' in redcap_variable:
        option = redcap_variable.split('(')[1].split(')')[0]
        if option 
            
            

form_df[form_df['redcap_variable'].str.contains('2018')]

,redcap_variable,value
66,q2018a,1
117,q2018(0),0
118,q2018(1),0
119,q2018(2),0
120,q2018(3),0
121,q2018(_10),1


In [46]:
# 1. record initialization
patient_name = '11003'
crf_name = 'mr'
form_df = pd.DataFrame(project.export_records(records=[patient_name], forms=[crf_name]))
form_df

,record_id,redcap_repeat_instrument,redcap_repeat_instance,q2001,q2002___1,q2002___2,q2002___3,q2002___4,q2003___1,q2003___2,...,q2119,q2121,q2122,q2124,q2129,q2131,mr_status,mri_comments,mr_init,mr_complete
0,11003,,,3,1,0,0,0,1,1,...,,,,,MGS,Only DWI sequence\r\n,4,"""Prior infarct seen on image?"" unanswered; JJJ...",JJJ,2


In [59]:
# 2. insert from logs


s = AMBRA_Backups.redcap_funcs.grab_logs(db, project, only_record_logs=False, start_date=datetime(2000, 1, 1))
s = pd.DataFrame(s)
sub = s[s['record'] == '11003']
display_md(sub)

|      | timestamp        | username   | action              | details                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |   record |
|-----:|:-----------------|:-----------|:--------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------

In [44]:
comp_schema_cap_db(db_name, project_name)

/Users/naderdl/Documents/AMBRA_Backups/AMBRA_Backups/redcap_funcs.py:283: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(row['field_label'], 'html.parser')
/Users/naderdl/Documents/AMBRA_Backups/AMBRA_Backups/redcap_funcs.py:283: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(row['field_label'], 'html.parser')
/Users/naderdl/Documents/AMBRA_Backups/AMBRA_Backups/redcap_funcs.py:283: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(row['field_label'], 'html.parser')



-------------------mr-------------------

The following CRF_Data_RedCap.redcap_variable's are not in CRF_schema_RedCap.redcap_variable's:
['q2018(-10)', 'q2029_mr(-10)', 'q1036_mr(3)']


The following api-metadata question_text's are not in CRF_Schema_RedCap.question_text's:
{redcap_variable : question_text}

{'q2018(-10)': 'White matter hyperintensity (WMH) Periventricular?', 'q2029_mr(-10)': 'White matter hyperintensity (WMH) deep white matter (DWM)?'}




Exception: Please handle the above discrepancies